In [2]:
from datasets import load_from_disk

# Specify the path to the dataset
dataset_path = '/home/sjoshi/lmm/data/generated/v3_spatial_grid'

# Load the dataset
dataset = load_from_disk(dataset_path)

# Display the dataset
print(dataset)

/home/sjoshi/anaconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})


In [3]:
import random 

print(dataset['train'][random.randint(0, 17)]["text"])

| bird | bird | automobile |
| automobile | deer | deer |
| bird | automobile | automobile |
The grid above is size 3 by 3. Each cell contains an object from ['automobile', 'deer', 'bird'].
What object is in row 0, column 2? A: automobile
What object is in row 1, column 2? A: deer
What object is in row 2, column 1? A: automobile
What object is in row 1, column 0? A: automobile
What object is in row 1, column 1? A: deer
What object is in row 2, column 2? A: automobile
What object is in row 0, column 1? A: bird
What object is in row 2, column 0? A: bird
What object is in row 0, column 0? A: bird


In [144]:
story = "One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt. Lily went to her mom and said, 'Mom, I found this needle. Can you share it with me and sew my shirt?' Her mom smiled and said, 'Yes, Lily, we can share the needle and fix your shirt.' Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together."

len(dataset['train'][random.randint(0, 17)]["text"])

586

In [39]:
# Specify the path to the new dataset
new_dataset_path = '/home/sjoshi/lmm/data/generated/v3_spatial_grid_multimodal'

# Load the new dataset
new_dataset = load_from_disk(new_dataset_path)

# Display the new dataset
print(new_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'image_1'],
        num_rows: 100000
    })
    validation: Dataset({
        features: ['text', 'image_1'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'image_1'],
        num_rows: 1000
    })
})


In [57]:
new_dataset["train"][0]["text"]

"| deer | cat | bird |\n| deer | cat | cat |\n| bird | dog | bird |\nThe grid above is size 3 by 3. Each cell contains an object from ['deer', 'bird', 'dog', 'cat'].\nWhat object is in row 2, column 2? A: bird\nWhat object is in row 1, column 2? A: cat\nWhat object is in row 0, column 0? A: deer\nWhat object is in row 0, column 1? A: cat\nWhat object is in row 1, column 0? A: deer\nWhat object is in row 1, column 1? A: cat\nWhat object is in row 0, column 2? A: bird\nWhat object is in row 2, column 1? A: dog\nWhat object is in row 2, column 0? A: bird"

In [58]:
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

def truncate_before_substring(text, substring):
    index = text.find(substring)
    if index != -1:
        return text[index:]
    return text

def make_multimodal_prompt(text):
    return "<image_1>\n" + truncate_before_substring(text, "The grid above")

fixed_dataset = {}
for split in ["train", "validation", "test"]:
    fixed_dataset[split] = []
    for i, example in enumerate(new_dataset[split]):
        example["text"] = make_multimodal_prompt(example["text"])
        fixed_dataset[split].append(example)
        
    # Create a dataframe from all examples for the current split
    df = pd.DataFrame(fixed_dataset[split])

    # Convert the dataframe to a HuggingFace dataset
    hf_dataset = Dataset.from_pandas(df)

    # Merge the new dataset with the existing dataset for the current split
    fixed_dataset[split] = hf_dataset
fixed_dataset = DatasetDict(fixed_dataset)
fixed_dataset.save_to_disk("/home/sjoshi/lmm/data/generated/v3_spatial_grid_multimodal")

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 481937.72 examples/s]
